<a href="https://colab.research.google.com/github/allen44/riiid-test-answer-prediction/blob/main/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%cd /content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/
%pwd

/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction


'/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction'

## Preprocessed data

In [2]:
import pickle
from pathlib import Path

# #Define data paths
df_train_preprocessed_path = Path('/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/data/intermediate/df_train_preprocessed.pkl.gzip')
df_lectures_preprocessed_path = Path('/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/data/intermediate/df_lectures_preprocessed.pkl.gzip')
df_questions_preprocessed_path = Path('/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/data/intermediate/df_questions_preprocessed.pkl.gzip')

Using our insights gained from the EDA, when can import the data from csv with the best preprocessing for feature engineering.

In [3]:
with open(df_train_preprocessed_path, 'rb') as f:
  df_train = pickle.load(f)

with open(df_lectures_preprocessed_path, 'rb') as f:
  df_lectures = pickle.load(f)

with open(df_questions_preprocessed_path, 'rb') as f:
  df_questions = pickle.load(f)

assert df_train['content_id'].dtype == df_lectures['lecture_id'].dtype
assert df_questions['question_id'].dtype == df_lectures['lecture_id'].dtype

df_train.shape, df_lectures.shape, df_questions.shape 

((101230331, 9), (418, 4), (13522, 192))

In [5]:
# Use a subset of df_train
df_train = df_train[df_train.index % 1000 == 0]
df_train.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,,,,
0,2018-01-02 03:03:47.951,115,5692,0,1,3,1,NaN,True
1000,2018-06-25 19:32:50.920,13134,9155,0,299,3,1,27666.0,True
2000,2018-01-20 21:54:35.915,24418,476,0,112,0,1,15000.0,True
3000,2018-02-26 01:28:59.702,24418,5168,0,1000,1,1,6000.0,True
4000,2018-03-08 03:28:44.125,24418,3746,0,1747,3,0,27000.0,True


## Manually add features

## Install and import featuretools

In [10]:
# % pip install featuretools dask distributed tornado
import featuretools as ft

# Prepare data

First, we specify a dictionary with all the entities in our dataset.

In [11]:
# Define entity set
es = ft.EntitySet()

# Add df_train to entity set
es.entity_from_dataframe(entity_id='train', 
                         dataframe=df_train,
                         index='row_id',
                         time_index='timestamp')

Entityset: None
  Entities:
    train [Rows: 101231, Columns: 10]
  Relationships:
    No relationships

In [12]:
# Add df_lectures to entity set
es.entity_from_dataframe(entity_id='lectures', dataframe=df_lectures, index='lecture_id')

Entityset: None
  Entities:
    train [Rows: 101231, Columns: 10]
    lectures [Rows: 418, Columns: 4]
  Relationships:
    No relationships

In [13]:
# Add df_questions to entity set
es.entity_from_dataframe(entity_id='questions', dataframe=df_questions, index='question_id')

Entityset: None
  Entities:
    train [Rows: 101231, Columns: 10]
    lectures [Rows: 418, Columns: 4]
    questions [Rows: 13522, Columns: 192]
  Relationships:
    No relationships

In [14]:
es['train'].variables

[<Variable: row_id (dtype = index)>,
 <Variable: timestamp (dtype: datetime_time_index, format: None)>,
 <Variable: user_id (dtype = categorical)>,
 <Variable: content_id (dtype = categorical)>,
 <Variable: content_type_id (dtype = categorical)>,
 <Variable: task_container_id (dtype = categorical)>,
 <Variable: user_answer (dtype = categorical)>,
 <Variable: answered_correctly (dtype = categorical)>,
 <Variable: prior_question_elapsed_time (dtype = numeric)>,
 <Variable: prior_question_had_explanation (dtype = boolean)>]

In [15]:
es['lectures'].variables

[<Variable: lecture_id (dtype = index)>,
 <Variable: tag (dtype = categorical)>,
 <Variable: part (dtype = categorical)>,
 <Variable: type_of (dtype = categorical)>]

In [16]:
es['questions'].variables

[<Variable: question_id (dtype = index)>,
 <Variable: bundle_id (dtype = categorical)>,
 <Variable: correct_answer (dtype = categorical)>,
 <Variable: part (dtype = categorical)>,
 <Variable: 0 (dtype = boolean)>,
 <Variable: 1 (dtype = boolean)>,
 <Variable: 10 (dtype = boolean)>,
 <Variable: 100 (dtype = boolean)>,
 <Variable: 101 (dtype = boolean)>,
 <Variable: 102 (dtype = boolean)>,
 <Variable: 103 (dtype = boolean)>,
 <Variable: 104 (dtype = boolean)>,
 <Variable: 105 (dtype = boolean)>,
 <Variable: 106 (dtype = boolean)>,
 <Variable: 107 (dtype = boolean)>,
 <Variable: 108 (dtype = boolean)>,
 <Variable: 109 (dtype = boolean)>,
 <Variable: 11 (dtype = boolean)>,
 <Variable: 110 (dtype = boolean)>,
 <Variable: 111 (dtype = boolean)>,
 <Variable: 112 (dtype = boolean)>,
 <Variable: 113 (dtype = boolean)>,
 <Variable: 114 (dtype = boolean)>,
 <Variable: 115 (dtype = boolean)>,
 <Variable: 116 (dtype = boolean)>,
 <Variable: 117 (dtype = boolean)>,
 <Variable: 118 (dtype = boolean)>

In [17]:
import gc

del df_train
del df_lectures
del df_questions

gc.collect()

153

In [18]:
r_lectures_train= ft.Relationship(es["lectures"]["lecture_id"],
                                          es["train"]["content_id"])

r_questions_train = ft.Relationship(es["questions"]["question_id"],
                                    es["train"]["content_id"])

es = es.add_relationship(r_lectures_train)
es = es.add_relationship(r_questions_train)
es

Entityset: None
  Entities:
    train [Rows: 101231, Columns: 10]
    lectures [Rows: 418, Columns: 4]
    questions [Rows: 13522, Columns: 192]
  Relationships:
    train.content_id -> lectures.lecture_id
    train.content_id -> questions.question_id

In [19]:
es['train']['answered_correctly'].interesting_values = [0, 1]

In [20]:
feature_defs = ft.dfs(entityset=es, target_entity='train', 
                      where_primitives = ['sum', 'mean'],
                      max_depth=2, features_only=True)

print(f'This will generate {len(feature_defs)} features.\n')

This will generate 242 features.



/usr/local/lib/python3.7/dist-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  where_primitives: ['mean', 'sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [21]:
import random; random.seed(42)

random.sample(feature_defs, 10)

[<Feature: questions.60>,
 <Feature: questions.107>,
 <Feature: prior_question_elapsed_time>,
 <Feature: questions.84>,
 <Feature: questions.145>,
 <Feature: questions.138>,
 <Feature: questions.133>,
 <Feature: questions.113>,
 <Feature: questions.83>,
 <Feature: questions.105>]

### Aggregation Primitives

In [ ]:
all_p = ft.list_primitives()
trans_p = all_p.loc[all_p['type'] == 'transform'].copy()
agg_p = all_p.loc[all_p['type'] == 'aggregation'].copy()

pd.options.display.max_colwidth = 100
# agg_p

In [23]:
# Specify aggregation primitives
agg_primitives = ['sum', 'time_since_last', 'avg_time_between', 'all', 'mode', 'num_unique', 'min', 'last', 
                  'mean', 'percent_true', 'max', 'std', 'count']

## Transform Primitives

In [ ]:
# trans_p

In [25]:
# Specify transformation primitives
trans_primitives = ['cum_sum', 'diff', 'time_since_previous']

### Where Primitives

These primitives are applied to the `interesting_values` to build conditional features. 

In [26]:
# Specify where primitives
where_primitives = ['sum', 'mean', 'percent_true', 'all', 'any']

## Custom Primitives

For this problem, I wrote a custom primitive that calculates the sum of a value in the month prior to the cutoff time.

The second custom primitive finds the time since a previous true value. It simply finds the time between True examples.

In [27]:
def total_previous_month(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    previous_month = time.month - 1
    year = time.year
   
    # Handle January
    if previous_month == 0:
        previous_month = 12
        year = time.year - 1
        
    # Filter data and sum up total
    df = df[(df['date'].dt.month == previous_month) & (df['date'].dt.year == year)]
    total = df['value'].sum()
    
    return total


def time_since_true(boolean, datetime):
    """Calculate time since previous true value"""
    
    if np.any(np.array(list(boolean)) == 1):
        # Create dataframe sorted from oldest to newest 
        df = pd.DataFrame({'value': boolean, 'date': datetime}).\
                sort_values('date', ascending = False).reset_index()

        older_date = None

        # Iterate through each date in reverse order
        for date in df.loc[df['value'] == 1, 'date']:

            # If there was no older true value
            if older_date == None:
                # Subset to times on or after true
                times_after_idx = df.loc[df['date'] >= date].index

            else:
                # Subset to times on or after true but before previous true
                times_after_idx = df.loc[(df['date'] >= date) & (df['date'] < older_date)].index
            older_date = date
            # Calculate time since previous true
            df.loc[times_after_idx, 'time_since_previous'] = (df.loc[times_after_idx, 'date'] - date).dt.total_seconds()

        return list(df['time_since_previous'])[::-1]
    
    # Handle case with no true values
    else:
        return [np.nan for _ in range(len(boolean))]

### Custom Primitive Implementation

Making a custom primitive is simple: first we define a function (`total_previous_month`) and then we `make_agg_primitive` with `input_type[s]`, a `return_type`, and whether or not the primitive requires the `cutoff_time` through `uses_calc_time`. 

This primitive is an aggregation primitive because it takes in multiple numbers - transactions for the previous month - and returns a single number - the total of the transactions. 

In [28]:
from featuretools.primitives import make_agg_primitive

# Takes in a number and outputs a number
total_previous = make_agg_primitive(total_previous_month, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)

In [29]:
from featuretools.primitives import make_trans_primitive

# Specify the inputs and return
time_since = make_trans_primitive(time_since_true, 
                                  input_types = [ft.variable_types.Boolean, 
                                                  ft.variable_types.Datetime],
                                  return_type = ft.variable_types.Numeric)

Now just have to pass this in as another aggregation primitive for Featuretools to use it in calculations.



Let's add the two custom primitives to the respective lists. In the final version of feature engineering, I did not use the `time_since` primitive. I ran into problems with the implementation but would encourage anyone to try and fix it or build their own custom primitive[s].

In [30]:
agg_primitives.append(total_previous)
trans_primitives.append(time_since)

## Deep Feature Synthesis with Specified Primitives

We'll again run Deep Feature Synthesis to make the feature definitions this time using the selected primitives and the custom primitives. 

In [31]:
feature_defs = ft.dfs(entityset=es, target_entity='train', 
                      # cutoff_time = cutoff_times, 
                      agg_primitives = agg_primitives,
                      trans_primitives = trans_primitives,
                      where_primitives = where_primitives,
                      chunk_size = 100, #len(cutoff_times), 
                      # cutoff_time_in_index = True,
                      max_depth = 2, 
                      features_only = True)

/usr/local/lib/python3.7/dist-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  where_primitives: ['all', 'any', 'mean', 'percent_true', 'sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [32]:
print(f'This will generate {len(feature_defs)} features.')

This will generate 452 features.


In [33]:
random.sample(feature_defs, 15)

[<Feature: TIME_SINCE_TRUE(questions.172, timestamp)>,
 <Feature: TIME_SINCE_TRUE(questions.33, timestamp)>,
 <Feature: TIME_SINCE_TRUE(questions.111, timestamp)>,
 <Feature: questions.121>,
 <Feature: TIME_SINCE_TRUE(questions.132, timestamp)>,
 <Feature: lectures.LAST(train.user_id)>,
 <Feature: questions.correct_answer>,
 <Feature: questions.bundle_id>,
 <Feature: questions.124>,
 <Feature: questions.182>,
 <Feature: questions.20>,
 <Feature: questions.NUM_UNIQUE(train.user_id)>,
 <Feature: TIME_SINCE_TRUE(questions.138, timestamp)>,
 <Feature: lectures.part>,
 <Feature: TIME_SINCE_TRUE(questions.119, timestamp)>]

# Run Deep Feature Synthesis

Once we're happy with the features that will be generated, we can run deep feature synthesis to make the actual features. We need to change `feature_only` to `False` and then we're good to go.

In [34]:
from timeit import default_timer as timer

start = timer()
feature_matrix, feature_defs = ft.dfs(entityset=es, 
                                      target_entity='train', 
                                      # cutoff_time = cutoff_times, 
                                      agg_primitives = agg_primitives,
                                      trans_primitives = trans_primitives,
                                      where_primitives = where_primitives,
                                      max_depth = 2, features_only = False,
                                      verbose = 1, 
                                      chunk_size = 100,  
                                      # n_jobs = -1,
                                      # cutoff_time_in_index = True
                                      )
end = timer()
print(f'{round(end - start)} seconds elapsed.')

Built 452 features
Elapsed: 00:00 | Progress:   0%|          

/usr/local/lib/python3.7/dist-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  where_primitives: ['all', 'any', 'mean', 'percent_true', 'sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Elapsed: 23:19 | Progress: 100%|██████████
1400 seconds elapsed.


The `chunk_size` is a parameter that may need to be adjusted to optimize the calculation. I suggest playing around with this parameter to find the optimal value. Generally I've found that a large value makes the calculation proceed quicker although it depends on the machine in use and the number of unique cutoff times. 

In [55]:
feature_matrix

,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,CUM_SUM(prior_question_elapsed_time),DIFF(prior_question_elapsed_time),TIME_SINCE_PREVIOUS(timestamp),"TIME_SINCE_TRUE(prior_question_had_explanation, timestamp)",lectures.tag,lectures.part,lectures.type_of,questions.bundle_id,questions.correct_answer,questions.part,questions.0,questions.1,questions.10,questions.100,questions.101,questions.102,questions.103,questions.104,questions.105,questions.106,questions.107,questions.108,questions.109,questions.11,questions.110,questions.111,questions.112,questions.113,questions.114,questions.115,questions.116,questions.117,...,"TIME_SINCE_TRUE(questions.63, timestamp)","TIME_SINCE_TRUE(questions.64, timestamp)","TIME_SINCE_TRUE(questions.65, timestamp)","TIME_SINCE_TRUE(questions.66, timestamp)","TIME_SINCE_TRUE(questions.67, timestamp)","TIME_SINCE_TRUE(questions.68, timestamp)","TIME_SINCE_TRUE(questions.69, timestamp)","TIME_SINCE_TRUE(questions.7, timestamp)","TIME_SINCE_TRUE(questions.70, timestamp)","TIME_SINCE_TRUE(questions.71, timestamp)","TIME_SINCE_TRUE(questions.72, timestamp)","TIME_SINCE_TRUE(questions.73, timestamp)","TIME_SINCE_TRUE(questions.74, timestamp)","TIME_SINCE_TRUE(questions.75, timestamp)","TIME_SINCE_TRUE(questions.76, timestamp)","TIME_SINCE_TRUE(questions.77, timestamp)","TIME_SINCE_TRUE(questions.78, timestamp)","TIME_SINCE_TRUE(questions.79, timestamp)","TIME_SINCE_TRUE(questions.8, timestamp)","TIME_SINCE_TRUE(questions.80, timestamp)","TIME_SINCE_TRUE(questions.81, timestamp)","TIME_SINCE_TRUE(questions.82, timestamp)","TIME_SINCE_TRUE(questions.83, timestamp)","TIME_SINCE_TRUE(questions.84, timestamp)","TIME_SINCE_TRUE(questions.85, timestamp)","TIME_SINCE_TRUE(questions.86, timestamp)","TIME_SINCE_TRUE(questions.87, timestamp)","TIME_SINCE_TRUE(questions.88, timestamp)","TIME_SINCE_TRUE(questions.89, timestamp)","TIME_SINCE_TRUE(questions.9, timestamp)","TIME_SINCE_TRUE(questions.90, timestamp)","TIME_SINCE_TRUE(questions.91, timestamp)","TIME_SINCE_TRUE(questions.92, timestamp)","TIME_SINCE_TRUE(questions.93, timestamp)","TIME_SINCE_TRUE(questions.94, timestamp)","TIME_SINCE_TRUE(questions.95, timestamp)","TIME_SINCE_TRUE(questions.96, timestamp)","TIME_SINCE_TRUE(questions.97, timestamp)","TIME_SINCE_TRUE(questions.98, timestamp)","TIME_SINCE_TRUE(questions.99, timestamp)"
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,115,5692,0,1,3,1,NaN,True,NaN,NaN,NaN,0.0,NaN,NaN,NaN,5692,3,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.000,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00,0.000,0.000,0.000000e+00,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000,NaN,0.000000e+00,0.000,0.000,0.000000e+00,NaN,0.000000e+00,0.000,NaN,NaN,NaN
164,3505219,7900,0,0,0,1,NaN,True,NaN,NaN,0.000,0.0,NaN,NaN,NaN,7900,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.000,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00,0.000,0.000,0.000000e+00,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000,NaN,0.000000e+00,0.000,0.000,0.000000e+00,NaN,0.000000e+00,0.000,NaN,NaN,NaN
451,9063293,5529,0,0,2,1,NaN,True,NaN,NaN,0.000,0.0,NaN,NaN,NaN,5529,2,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.000,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00,0.000,0.000,0.000000e+00,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000,NaN,0.000000e+00,0.000,0.000,0.000000e+00,NaN,0.000000e+00,0.000,NaN,NaN,NaN
526,10501741,7900,0,0,0,1,NaN,True,NaN,NaN,0.000,0.0,NaN,NaN,

We can save these feature definitions as a binary file which will allow us to make the same exact features for another entityset of the same format. This is useful when we have multiple partitions and we want to make the same features for each. Instead of remaking the feature definitions, we pass in the same feature definitions to a call to `calculate_feature_matrix`.

In [ ]:
feature_defs = ft.load_features('./data/churn/features.txt')
print(f'There are {len(feature_defs)} features.')

# Conclusions

Automated feature engineering is a significant improvement over manual feature engineering in terms of both time and modeling performance. In this notebook, we implemented an automated feature engineering workflow with Featuretools for the customer churn problem. Given customer data and label times, we can now calculate a feature matrix with several hundred relevant features for predicting customer churn while ensuring that our features are made with valid data for each cutoff time. 

Along the way, we implemented a number of Featuretools concepts:

1. An entityset and entities
2. Relationships between entities
3. Cutoff times
4. Feature primitives
5. Custom primitives
6. Deep feature synthesis

These concepts will serve us well in future machine learning projects that we can tackle with automated feature engineering.

## Next Steps

Although we often hear that "data is the fuel of machine learning", data is not exactly a fuel but more like crude oil. _Features_ are the refined product that we feed into a machine learning model to make accurate predictions. After performing prediction engineering and automated feature engineering, the next step is to use these features in a predictive model to estimate the _label_ using the _features_. 

Generating hundreds of features automatically is impressive, but if those features cannot allow a model to learn our prediction problem then they are not mcuch help! The next step is to use our features and labeled historical examples to train a machine learning model to make predictions of customer churn. We'll make sure to test our model using a hold-out testing set to estimate performance on new data. Then, after validating our model, we can use it on new examples by passing the data through the feature engineering process. 


If you want to see how to parallelize feature engineering in Spark, see the `Feature Engineering on Spark` notebook. Otherwise, the next notebook is `Modeling`, where we develop a machine learning model to predict churn using the historical labeled examples and the automatically engineered features.